In [1]:
import glob
import cv2
import numpy as np

def mean_RoI_size(mask_files) -> int:
    dimensions = []

    for mask_file in mask_files:
        mask = cv2.imread(mask_file, 0)
        mask = mask / 255
        mask = mask.astype(np.uint8)

        # Find the contours in the mask
        contours, _ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)

        for contour in contours:
            # Get the bounding rectangle of the contour and append the maximum dimension to the list
            rect = cv2.boundingRect(contour)
            height = rect[3]
            width = rect[2]
            dimensions.append(max(height, width))

    return int(np.mean(dimensions))

In [2]:
def image_contrast_level(img: np.ndarray, mask: np.ndarray) -> float:
    RoI = img[mask == 1].flatten()
    not_RoI = img[mask == 0].flatten()

    if len(RoI) == 0 or len(not_RoI) == 0:
        return None

    RoI_hist_norm = np.histogram(RoI, bins=10, range=(0, 256))[
        0] / len(RoI)
    not_RoI_hist_norm = np.histogram(not_RoI, bins=10, range=(0, 256))[
        0] / len(not_RoI)
    contrast = np.mean(np.abs(RoI_hist_norm - not_RoI_hist_norm))
    return contrast

In [18]:
def mean_contrast_level(imgs_dir: str, masks_dir: str, file_type: str = 'png') -> float:
    img_files = imgs_dir
    img_files.sort()
    mask_files = masks_dir
    mask_files.sort()
    files = list(zip(img_files, mask_files))

    contrasts = []

    for img_file, mask_file in files:
        img = cv2.imread(img_file, 0)
        mask = cv2.imread(mask_file, 0)
        mask = mask / 255
        mask = mask.astype(np.uint8)

        contrast = image_contrast_level(img, mask)

        if contrast is not None:
            contrasts.append(contrast)

    return np.mean(contrasts)

In [19]:
from utils.data import ALL_DATASETS

datasets = list(ALL_DATASETS.keys())
datasets.remove("dummy")

for dataset in datasets:
    mask_files = glob.glob(f"data/preprocessed/{dataset}/train/masks/*.png")
    mask_files += glob.glob(f"data/preprocessed/{dataset}/val/masks/*.png")
    mask_files += glob.glob(f"data/preprocessed/{dataset}/test/masks/*.png")

    print(f"{dataset}: {mean_RoI_size(mask_files)}")

fetal_head: 260
fetal_head_2: 255
kidney: 101
lungs: 329
thyroid: 187
nerve: 159
shapes_a: 168
shapes_b: 168
shapes_a_large: 244
shapes_b_large: 238
shapes_a_contour: 169
shapes_b_contour: 168
shapes_a_large_contour: 243
shapes_b_large_contour: 237


In [20]:
from utils.data import ALL_DATASETS

datasets = list(ALL_DATASETS.keys())
datasets.remove("dummy")

for dataset in datasets:
    image_files = glob.glob(f"data/preprocessed/{dataset}/train/*.png")
    image_files += glob.glob(f"data/preprocessed/{dataset}/val/*.png")
    image_files += glob.glob(f"data/preprocessed/{dataset}/test/*.png")
    mask_files = glob.glob(f"data/preprocessed/{dataset}/train/masks/*.png")
    mask_files += glob.glob(f"data/preprocessed/{dataset}/val/masks/*.png")
    mask_files += glob.glob(f"data/preprocessed/{dataset}/test/masks/*.png")

    # print(len(image_files), len(mask_files))

    print(f"{dataset}: {mean_contrast_level(image_files, mask_files)}")

fetal_head: 0.059890137215260965
fetal_head_2: 0.06653948649256097
kidney: 0.16664373959384315
lungs: 0.13100204956874126
thyroid: 0.14964160960773834
nerve: 0.048517518210639855
shapes_a: 0.19969839424562874
shapes_b: 0.19195257843231986
shapes_a_large: 0.19934751974983594
shapes_b_large: 0.1800119698366292
shapes_a_contour: 0.014571840703483541
shapes_b_contour: 0.013416561243448751
shapes_a_large_contour: 0.005176048223929
shapes_b_large_contour: 0.005445086126375616
